In [23]:
import findspark

findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

import pandas as pd

from pyspark.sql.functions import isnan, when, count

In [30]:
df = spark.read.csv("nuclear_plants_small_dataset.csv", inferSchema=True, header=True)
df = df.toDF('Status', 'PoRS1', 'PoRS2', 'PoRS3', 'PoRS4', 'PrS1', 'PrS2', 'PrS3', 'PrS4', 'VS1', 'VS2', 'VS3', 'VS4')

In [31]:
df.printSchema()
df.count()

root
 |-- Status: string (nullable = true)
 |-- PoRS1: double (nullable = true)
 |-- PoRS2: double (nullable = true)
 |-- PoRS3: double (nullable = true)
 |-- PoRS4: double (nullable = true)
 |-- PrS1: double (nullable = true)
 |-- PrS2: double (nullable = true)
 |-- PrS3: double (nullable = true)
 |-- PrS4: double (nullable = true)
 |-- VS1: double (nullable = true)
 |-- VS2: double (nullable = true)
 |-- VS3: double (nullable = true)
 |-- VS4: double (nullable = true)



996

In [35]:
# Count non-numbers in all columns
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------+-----+-----+-----+-----+----+----+----+----+---+---+---+---+
|Status|PoRS1|PoRS2|PoRS3|PoRS4|PrS1|PrS2|PrS3|PrS4|VS1|VS2|VS3|VS4|
+------+-----+-----+-----+-----+----+----+----+----+---+---+---+---+
|     0|    0|    0|    0|    0|   0|   0|   0|   0|  0|  0|  0|  0|
+------+-----+-----+-----+-----+----+----+----+----+---+---+---+---+

